In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
en = cptac.Endometrial()
br = cptac.Brca()
cl = cptac.Colon()

In [3]:
gene = 'PIK3CA'

# Endometrial

In [24]:
mut = en.get_genotype_all_vars(gene, mutation_hotspot=['E542K', 'E545K', 'H1047R'])
phos = en.get_phosphoproteomics(tissue_type="tumor")

joined = mut.join(phos)
joined['Mutation'].unique()

#select samples containing hotspot mutations
en_hotspot_df = joined[joined.Location.str.contains('E542K') | 
                    joined.Location.str.contains('E545K') |
                    joined.Location.str.contains('H1047R')]
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]

endo_hotspot = en_hotspot_df.append(wildtype)
endo_hotspot = endo_hotspot.drop(columns = ["Mutation_Status", "Location"])


# Keep two values to compare
compare = ['Wildtype_Tumor','Missense_Mutation']
get = endo_hotspot['Mutation'].isin(compare)
missense_wt = endo_hotspot[get]
missense_wt['Mutation'].value_counts()


# Step 2 get the difference in medians
missense = missense_wt[missense_wt.Mutation == "Missense_Mutation"] #get all missense_mutations
wt = missense_wt[missense_wt.Mutation == "Wildtype_Tumor"] #getn all wildtype
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype
missense

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 48 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


,Mutation,"(AAAS, S495)","(AAAS, S541)","(AAAS, Y485)","(AACS, S618)","(AAED1, S12)","(AAGAB, S310)","(AAGAB, S311)","(AAK1, S14)","(AAK1, S18)",...,"(ZZZ3, S397)","(ZZZ3, S411)","(ZZZ3, S420)","(ZZZ3, S424)","(ZZZ3, S426)","(ZZZ3, S468)","(ZZZ3, S89)","(ZZZ3, T415)","(ZZZ3, T418)","(ZZZ3, Y399)"
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00006,Missense_Mutation,NaN,NaN,NaN,-0.8810,-1.8100,NaN,NaN,NaN,-0.2420,...,0.1840,NaN,NaN,NaN,-0.20500,NaN,NaN,NaN,NaN,NaN
C3L-00032,Missense_Mutation,-0.20200,NaN,NaN,NaN,-1.8800,NaN,NaN,NaN,0.3820,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C3L-00362,Missense_Mutation,NaN,NaN,NaN,0.8440,0.2790,NaN,NaN,NaN,-0.3550,...,0.0805,NaN,NaN,NaN,-0.10900,NaN,NaN,NaN,NaN,NaN
C3L-00601,Missense_Mutation,0.59400,NaN,NaN,0.0000,-0.9820,-0.276,NaN,NaN,-0.1090,...,0.1340,NaN,NaN,0.3200,0.32000,NaN,NaN,NaN,NaN,NaN
C3L-00605,Missense_Mutation,-0.16200,NaN,0.14,NaN,NaN,NaN,NaN,NaN,NaN,...,0.1890,NaN,NaN,NaN,0.38600,NaN,NaN,NaN,NaN,0.5960
C3L-00921,Missense_Mutation,-0.52500,NaN,NaN,0.8940,-1.4000,NaN,-0.255,0.271,0.7610,...,-0.1440,NaN,NaN,NaN,-0.22000,NaN,NaN,0.336,NaN,-0.1310
C3L-00947,Missense_Mutation,0.46000,NaN,NaN,NaN,-0.2650,NaN,NaN,NaN,0.6080,...,0.4830,NaN,NaN,NaN,0.48300,NaN,NaN,NaN,NaN,NaN
C3N-00323,Missense_Mutation,0.53200,NaN,NaN,NaN,-0.0342,NaN,NaN,NaN,-0.4470,...,0.6280,NaN,0.2,-0.8060,-0.80600,NaN,NaN,NaN,0.635,0.2000
C3N-00324,Missense_Mutation,-0.79100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2810,...,-0.3435,NaN,NaN,NaN,-0.09940,NaN,NaN,NaN,NaN,NaN


In [23]:
endo_d = {}

for prot in phos:
    dif_endo = wt_med[prot] - missense_med[prot]
    endo_d[prot] = dif_endo
#     print(wt_med[prot], '-', missense_med[prot], '=',dif_endo)

median_diff = pd.DataFrame.from_dict(endo_d, orient='index', columns=['Difference_In_Median'])

#step 3 do a t test for every gene and report the p-value
cols = list(missense_wt.columns[1:])
t_test = u.wrap_ttest(missense_wt, 'Mutation', cols, return_all = True)
t_test = t_test.set_index('Comparison')

# Step 4 join median and p value together
endo_df = median_diff.join(t_test)
endo_df

/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/stats/multitest.py:153: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals <= alphacBonf
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/stats/multitest.py:257: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1


,Difference_In_Median,P_Value
"(AAAS, S495)",0.257295,0.650738
"(AAAS, S541)",-0.061000,NaN
"(AAAS, Y485)",-0.300000,NaN
"(AACS, S618)",-0.019150,0.980243
"(AAED1, S12)",0.378000,0.168105
...,...,...
"(ZZZ3, S468)",NaN,NaN
"(ZZZ3, S89)",NaN,NaN
"(ZZZ3, T415)",-0.031250,NaN
"(ZZZ3, T418)",-0.725175,NaN


# Brca

In [27]:
mut = br.get_genotype_all_vars(gene, mutation_hotspot=['E542K', 'E545K', 'H1047R'])
phos = br.get_phosphoproteomics(tissue_type="tumor")
phos

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 83 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)


Name                A2M                       AAAS  \
Site               S710                       S495   
Peptide     VGFYEsDVMGR     IAHIPLYFVNAQFPRFsPVLGR   
Database_ID NP_000005.2 NP_056480.1|NP_001166937.1   
CPT000814           NaN                     1.9431   
CPT001846           NaN                     0.2274   
X01BR001            NaN                    -2.2853   
X01BR008            NaN                     1.3714   
X01BR009            NaN                     0.2682   
...                 ...                        ...   
X21BR001            NaN                     0.9424   
X21BR002            NaN                     0.2955   
X21BR010        -0.7266                    -0.1201   
X22BR005        -0.7618                    -0.3378   
X22BR006            NaN                    -0.9958   

Name                                                        \
Site                                                  S541   
Peptide     AQEPPAGGGGSIHDLPLFTETSPTSAPWDPLPGPPPVLPHsPHSHL   
Database_ID                     NP_056480.1|NP_001166937.1   
CPT000814                                              NaN   
CPT001846                                              NaN   
X01BR001                                           -0.8967   
X01BR008                                               NaN   
X01BR009                                               NaN   
...                                                    ...   
X21BR001                                               NaN   
X21BR002                                               NaN   
X21BR010                                           -0.1477   
X22BR005                                            3.2472   
X22BR006                                               NaN   

Name                                     AAED1                      AAGAB  \
Site                                       S12                   S310S311   
Peptide     QVsGAAALVPAPSGPDSGQPLAAAVAELPVLDAR     AFWMAIGGDRDEIEGLssDEEH   
Database_ID                        NP_714542.1 NP_078942.3|NP_001258815.1   
CPT000814                                  NaN                     0.0127   
CPT001846                               0.5472                     0.6895   
X01BR001                                4.1225                     0.4842   
X01BR008                               -0.0636                    -0.6224   
X01BR009                                   NaN                    -0.6207   
...                                        ...                        ...   
X21BR001                                0.4838                     0.0477   
X21BR002                               -0.0053                     0.1169   
X21BR010                                1.8818                     0.8338   
X22BR005                                   NaN                     1.0000   
X22BR006                               -0.4590                    -1.0934   

Name                                                           AAK1  \
Site                              S311                          S14   
Peptide         AFWMAIGGDRDEIEGLSsDEEH EQGGsGLGSGSSGGGGSTSGLGSGYIGR   
Database_ID NP_078942.3|NP_001258815.1                  NP_055726.3   
CPT000814                      -0.4495                          NaN   
CPT001846                      -0.7680                          NaN   
X01BR001                       -1.2458                      -0.0310   
X01BR008                       -1.2732                      -1.1351   
X01BR009                       -2.3561                      -0.3078   
...                                ...                          ...   
X21BR001                       -0.3365                      -0.4697   
X21BR002                        0.5654                      -0.7218   
X21BR010                        0.2280                      -0.0877   
X22BR005                        1.8410                          NaN   
X22BR006                        1.3237                          NaN   

Name                                

In [42]:
fil = phos.columns.get_level_values('Site').duplicated(False)
duplicates = phos[phos.columns[fil]]
len(duplicates.columns.get_level_values('Site').unique())

3380

In [39]:
duplicates = phos[phos.columns[~fil]]
len(duplicates.columns.get_level_values('Peptide').unique())

38731

In [ ]:
joined = mut.join(phos)
joined['Mutation'].unique()

#select samples containing hotspot mutations
br_hotspot_df = joined[joined.Location.str.contains('E542K') | 
                    joined.Location.str.contains('E545K') |
                    joined.Location.str.contains('H1047R')]
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]

brca_hotspot = br_hotspot_df.append(wildtype)
brca_hotspot = brca_hotspot.drop(columns = ["Mutation_Status", "Location"])


# Keep two values to compare
compare = ['Wildtype_Tumor','Missense_Mutation']
get = brca_hotspot['Mutation'].isin(compare)
missense_wt = brca_hotspot[get]
missense_wt['Mutation'].value_counts()


# Step 2 get the difference in medians
missense = missense_wt[missense_wt.Mutation == "Missense_Mutation"] #get all missense_mutations
wt = missense_wt[missense_wt.Mutation == "Wildtype_Tumor"] #getn all wildtype
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype
missense


brca_d = {}

for prot in phos:
    dif_brca = wt_med[prot] - missense_med[prot]
    brca_d[prot] = dif_brca
#     print(wt_med[prot], '-', missense_med[prot], '=',dif_endo)

median_diff = pd.DataFrame.from_dict(brca_d, orient='index', columns=['Difference_In_Median'])

#step 3 do a t test for every gene and report the p-value
cols = list(missense_wt.columns[1:])
t_test = u.wrap_ttest(missense_wt, 'Mutation', cols, return_all = True)
t_test = t_test.set_index('Comparison')

# Step 4 join median and p value together
brca_df = median_diff.join(t_test)
brca_df